In [ ]:
#!pip install telethon

In [ ]:
# Initial imports
from datetime import datetime, timezone
import pandas as pd
import time
import json
import re

# Telegram imports
from telethon.sync import TelegramClient

# Setup / change only the first time you use it
username = 'roskameyka'  # Your Telegram account username (just 'abc123', not '@')
phone = '+31616045195'  # Your Telegram account phone number (ex: '+5511999999999')
api_id = '15331462'  # Your API ID, it can be only generated from https://my.telegram.org/apps
api_hash = '0ea1021965ce6dd5aa5370b94abc9c2d'  # Your API hash, also from https://my.telegram.org/apps

In [ ]:
# Setup / change every time to define scraping parameters
channels = [
    '@moscowmap', #Moscow
    '@moscowach', #Moscow
    '@piterach' #Saint Petersburg
    '@ngs_news', #Novosibirsk
    '@novostisochi', #Sochi
    '@RU66RU', #Yekaterinburg
    '@ngs24_krsk', #Krasnoyarsk
    '@rian_ru' #National
]
# Here you put the name of the channel or group that you want to scrape
# As an example, play: '@LulanoTelegram' or 'https://t.me/LulanoTelegram'
# Do not use: 'https://web.telegram.org/a/#-1001249230829' or '-1001249230829'

d_min = 1  # Start day (inclusive)
m_min = 1  # Start month
y_min = 2000  # Start year
d_max = 19  # End day (exclusive)
m_max = 11  # End month
y_max = 2024  # End year
key_search = 'дтп'  # Keyword to search, leave empty if not needed
max_t_index = 10  # Maximum number of messages to scrape
time_limit = 6 * 60 * 60  # Timeout in hours (*seconds)

In [ ]:
data = []  # List to store scraped data
t_index = 0  # Tracker for the number of messages processed
start_time = time.time()  # Record the start time for the scraping session

# Function to remove invalid XML characters from text
def remove_unsupported_characters(text):
    valid_xml_chars = (
        "[^\u0009\u000A\u000D\u0020-\uD7FF\uE000-\uFFFD"
        "\U00010000-\U0010FFFF]"
    )
    cleaned_text = re.sub(valid_xml_chars, '', text)
    return cleaned_text

# Function to format time in days, hours, minutes, and seconds
def format_time(seconds):
    days = seconds // 86400
    hours = (seconds % 86400) // 3600
    minutes = (seconds % 3600) // 60
    seconds = seconds % 60
    return f'{int(days):02}:{int(hours):02}:{int(minutes):02}:{int(seconds):02}'

# Function to print progress of the scraping process
def print_progress(t_index, message_id, start_time, max_t_index):
    elapsed_time = time.time() - start_time
    current_progress = t_index / (t_index + message_id) if (t_index + message_id) <= max_t_index else t_index / max_t_index
    percentage = current_progress * 100
    estimated_total_time = elapsed_time / current_progress
    remaining_time = estimated_total_time - elapsed_time

    elapsed_time_str = format_time(elapsed_time)
    remaining_time_str = format_time(remaining_time)

    print(f'Progress: {percentage:.2f}% | Elapsed Time: {elapsed_time_str} | Remaining Time: {remaining_time_str}')

# Scraping process
for channel in channels:
    if t_index >= max_t_index:
        break

    if time.time() - start_time > time_limit:
        break

    loop_start_time = time.time()

    try:
        c_index = 0
        async with TelegramClient(username, api_id, api_hash) as client:
            async for message in client.iter_messages(channel, search=key_search):
                try:
                    if datetime(y_min, m_min, d_min, tzinfo=timezone.utc) < message.date <= datetime(y_max, m_max, d_max, tzinfo=timezone.utc):

                        # Process comments of the message
                        comments_list = []
                        try:
                            async for comment_message in client.iter_messages(channel, reply_to=message.id):
                                comment_text = comment_message.text.replace("'", '"')

                                comment_media = 'True' if comment_message.media else 'False'

                                comment_emoji_string = ''
                                if comment_message.reactions:
                                    for reaction_count in comment_message.reactions.results:
                                        emoji = reaction_count.reaction.emoticon
                                        count = str(reaction_count.count)
                                        comment_emoji_string += emoji + " " + count + " "

                                comment_date_time = comment_message.date.strftime('%Y-%m-%d %H:%M:%S')

                                comments_list.append({
                                    'Type': 'comment',
                                    'Comment Group': channel,
                                    'Comment Author ID': comment_message.sender_id,
                                    'Comment Content': comment_text,
                                    'Comment Date': comment_date_time,
                                    'Comment Message ID': comment_message.id,
                                    'Comment Author': comment_message.post_author,
                                    'Comment Views': comment_message.views,
                                    'Comment Reactions': comment_emoji_string,
                                    'Comment Shares': comment_message.forwards,
                                    'Comment Media': comment_media,
                                    'Comment Url': f'https://t.me/{channel}/{message.id}?comment={comment_message.id}'.replace('@', ''),
                                })
                        except Exception as e:
                            comments_list = []
                            print(f'Error processing comments: {e}')

                        # Process the main message
                        media = 'True' if message.media else 'False'

                        emoji_string = ''
                        if message.reactions:
                            for reaction_count in message.reactions.results:
                                emoji = reaction_count.reaction.emoticon
                                count = str(reaction_count.count)
                                emoji_string += emoji + " " + count + " "

                        date_time = message.date.strftime('%Y-%m-%d %H:%M:%S')
                        cleaned_content = remove_unsupported_characters(message.text)
                        cleaned_comments_list = remove_unsupported_characters(json.dumps(comments_list))

                        data.append({
                            'Type': 'text',
                            'Group': channel,
                            'Author ID': message.sender_id,
                            'Content': cleaned_content,
                            'Date': date_time,
                            'Message ID': message.id,
                            'Author': message.post_author,
                            'Views': message.views,
                            'Reactions': emoji_string,
                            'Shares': message.forwards,
                            'Media': media,
                            'Url': f'https://t.me/{channel}/{message.id}'.replace('@', ''),
                            'Comments List': cleaned_comments_list,
                        })

                        c_index += 1
                        t_index += 1

                        # Print progress
                        print(f'{"-" * 80}')
                        print_progress(t_index, message.id, start_time, max_t_index)
                        current_max_id = min(c_index + message.id, max_t_index)
                        print(f'From {channel}: {c_index:05} contents of {current_max_id:05}')
                        print(f'Id: {message.id:05} / Date: {date_time}')
                        print(f'Total: {t_index:05} contents until now')
                        print(f'{"-" * 80}\n\n')

                        if t_index >= max_t_index:
                            break

                        if time.time() - start_time > time_limit:
                            break

                    elif message.date < datetime(y_min, m_min, d_min, tzinfo=timezone.utc):
                        break

                except Exception as e:
                    print(f'Error processing message: {e}')

        print(f'\n\n##### {channel} was ok with {c_index:05} posts #####\n\n')

        df = pd.DataFrame(data)

    except Exception as e:
        print(f'{channel} error: {e}')

    loop_end_time = time.time()
    loop_duration = loop_end_time - loop_start_time

    if loop_duration < 60:
        time.sleep(60 - loop_duration)

print(f'\n{"-" * 50}\n#Concluded! #{t_index:05} posts were scraped!\n{"-" * 50}\n\n\n\n')
df = pd.DataFrame(data)

In [ ]:
df